https://www.geeksforgeeks.org/microsoft-stock-price-prediction-with-machine-learning/



In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics

In [2]:
import sys
import logging

sys.path.append(r'\Users\jimhu\option_trader\src')

from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.settings.logConfig import LOGGING

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("urllib3").setLevel(logging.WARNING)


LOGGING['loggers']['']['level'] = 'DEBUG'

logging.config.dictConfig(LOGGING)
    

     

ModuleNotFoundError: No module named 'tradingBot'

In [ ]:
symbol = 'MSFT'
period = '1d'
interval = '1d'

df = get_price_history(symbol, period=period, interval=interval)
current_price = df['Close'].iloc[-1]
print(current_price)    

In [ ]:
#############################################################
#
#                  CALL/PUT OPTION CHAIN
#
############################################################

import sys

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.consts import optionQuote as oq
from tradingBot.utils.dataGetter import get_option_chain

max_strike_ratio = 0.25
max_days_to_expire = 720
min_open_interest=100
min_price=1.5
max_bid_ask_spread=3

symbol = 'MSFT'
call_chain = get_option_chain(symbol, 
                at.CALL, 
                exp_date_list = [], 
                min_price = min_price,
                max_days_to_expire=max_days_to_expire,                     
                max_bid_ask_spread=max_bid_ask_spread,
                min_open_interest=min_open_interest) 

                
put_chain = get_option_chain(symbol, 
                at.PUT, 
                exp_date_list = [], 
                min_price = min_price,
                max_days_to_expire=max_days_to_expire,                     
                max_bid_ask_spread=max_bid_ask_spread,
                min_open_interest=min_open_interest) 

def dump_chain(chain):
    print('----------')    
    print(chain[[oq.EXP_DATE, oq.STRIKE, oq.PRICE, oq.WIN_PROB_SHORT, oq.WIN_PROB_LONG, oq.BID_ASK_SPREAD, oq.IMPLIED_VOLATILITY, oq.DELTA]])
 

exp_date_list = list(set(put_chain[oq.EXP_DATE]))
exp_date_list.sort()

#display(put_chain, exp_date_list[0])

chain = call_chain
exp_date=exp_date_list[0]

dump_chain(call_chain.sort_values(oq.EXP_DATE))
dump_chain(put_chain.sort_values(oq.EXP_DATE))


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("urllib3").setLevel(logging.WARNING)

call_chain = get_option_chain(symbol, at.CALL, max_strike_ratio=0.2) 
           
put_chain = get_option_chain(symbol, at.PUT) 

In [ ]:
#https://www.geeksforgeeks.org/three-dimensional-plotting-in-python-using-matplotlib/
#https://realpython.com/pandas-plot-python/
#https://www.investopedia.com/articles/optioninvestor/11/predict-earnings-with-options.asp

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')

from tradingBot.consts import optionQuote as oq

features = [oq.DAYS_TO_EXPIRE, oq.STRIKE, oq.BID_ASK_SPREAD, oq.IMPLIED_VOLATILITY, oq.OPEN_INTEREST, oq.VOLUME]
 

plt.subplots(figsize=(20,10))
print(symbol + '----- CALL --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(call_chain[col])
plt.show()


plt.subplots(figsize=(20,10))
print(symbol + '----- PUT --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(put_chain[col])
plt.show()

In [ ]:
# importing packages
import seaborn as sb
import matplotlib.pyplot as plt

features = [oq.IN_MONEY, oq.STRIKE, oq.IMPLIED_VOLATILITY, oq.OPEN_INTEREST, oq.VOLUME, oq.DELTA]

c = call_chain[call_chain[oq.EXP_DATE]==exp_date_list[0]]
c = c[features]

p = put_chain[put_chain[oq.EXP_DATE]==exp_date_list[0]]
p = p[features]

print('----- CALL -----')
sb.pairplot(c, hue =oq.IN_MONEY)
plt.show()

print('----- PUT -----')
sb.pairplot(p, hue =oq.IN_MONEY)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tradingBot.settings.logConfig import LOGGING
logging.getLogger("matplotlib").setLevel(logging.WARNING)

c = call_chain #[call_chain[oq.EXP_DATE]==exp_date_list[0]]
p = put_chain #[call_chain[oq.EXP_DATE]==exp_date_list[0]]

x1 = c[oq.DAYS_TO_EXPIRE]
y1 = c[oq.OPEN_INTEREST]
#print(symbol+'---- CALL-----')
x2 = p[oq.DAYS_TO_EXPIRE]
y2 = p[oq.OPEN_INTEREST]

fig, (ax1, ax2) = plt.subplots(2, 1)
#fig.suptitle(oq.OPEN_INTEREST)

ax1.plot(x1, y1, '.-')
ax1.set_ylabel(oq.OPEN_INTEREST)

ax2.plot(x2, y2, '.-')
ax2.set_xlabel(oq.DAYS_TO_EXPIRE)
ax2.set_ylabel(oq.OPEN_INTEREST)



In [ ]:
c = call_chain[call_chain[oq.EXP_DATE]==exp_date_list[0]]
c

print(symbol+'---- CALL-----')
c.plot(x=oq.STRIKE, y=oq.OPEN_INTEREST, figsize=(3,1))
plt.axvline(x=current_price, color='r', ls='--', lw=2)
c.plot(x=oq.STRIKE, y=oq.VOLUME, figsize=(3,1))
plt.axvline(x=current_price, color='r', ls='--', lw=2)
plt.show()

p = put_chain[put_chain[oq.EXP_DATE]==exp_date_list[0]]
p
print(symbol+'---- PUT-----')

p.plot(x=oq.STRIKE, y=oq.OPEN_INTEREST, figsize=(3,1))
plt.axvline(x=current_price, color='r', ls='--', lw=2)
p.plot(x=oq.STRIKE, y=oq.VOLUME, figsize=(3,1))
plt.axvline(x=current_price, color='r', ls='--', lw=2)

In [ ]:
call_chain[oq.BID_ASK_SPREAD].plot(kind="hist",figsize=(3,1))
plt.show()
put_chain[oq.BID_ASK_SPREAD].plot(kind="hist",figsize=(3,1))

In [ ]:
df = call_chain

df.plot(x=oq.DAYS_TO_EXPIRE, y=oq.BID_ASK_SPREAD, kind="scatter", figsize=(3,1))
df.plot(x=oq.IN_MONEY, y=oq.BID_ASK_SPREAD, kind="scatter", figsize=(3,1))
df.plot(x=oq.IMPLIED_VOLATILITY, y=oq.BID_ASK_SPREAD, kind="scatter", figsize=(3,1))
df.plot(x=oq.STRIKE, y=oq.BID_ASK_SPREAD, kind="scatter",figsize=(3,1))



In [ ]:

# importing mplot3d toolkits, numpy and matplotlib
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
 
fig = plt.figure()
 
# syntax for 3-D projection
ax = plt.axes(projection ='3d')
 
df = put_chain

# defining all 3 axis
z = df[oq.DELTA]
x = df[oq.IN_MONEY]
y = df[oq.DAYS_TO_EXPIRE]
 
# plotting
ax.plot3D(x, y, z, 'green')
ax.set_title('3D line plot geeks for geeks')
ax.set_xlabel(oq.IN_MONEY, fontsize=10)
ax.set_ylabel(oq.DAYS_TO_EXPIRE, fontsize=10)
ax.set_zlabel(oq.PRICE, fontsize=10)
plt.show()

In [ ]:

#############################################################
#
#                  PICK LONG CALL/PUT
#
############################################################

import sys

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.consts import optionQuote as oq
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_option_long
from tradingBot.settings.logConfig import LOGGING
LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)

def dump_chain(chain): 
    print(chain[[oq.EXP_DATE, oq.STRIKE, oq.PRICE, oq.PROFIT, oq.PNL, oq.WIN_PROB, oq.IMPLIED_VOLATILITY, oq.DELTA,oq.OPEN_INTEREST, oq.BID_ASK_SPREAD]])
 

symbol = 'MSFT'

otype = at.CALL

target_price = get_price_history(symbol)['Close'].iloc[-1]

long_call = pick_option_long(symbol, at.CALL,
                target_price,            
                min_annualized_return=0,
                min_pnl = 0.4,
                max_IV = 0.5,
                max_bid_ask_spread=3,
                max_days_to_expire=720, 
                min_open_interest=100)

long_put = pick_option_long(symbol, at.PUT,
                target_price * 0.95,            
                min_annualized_return=0,
                min_pnl = 0.4,
                max_IV = 0.5,
                max_bid_ask_spread=3,
                max_days_to_expire=720, 
                min_open_interest=100)

print('#call', long_call.shape[0], ' #put ', long_put.shape[0])

print('------CALL----')   
if long_call.shape[0] > 0:
    dump_chain(long_call.sort_values(oq.EXP_DATE))
print('------PUT----')       
if long_put.shape[0] > 0:
    dump_chain(long_put.sort_values(oq.EXP_DATE))

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')

features = [oq.DAYS_TO_EXPIRE, oq.PRICE, oq.PROFIT, oq.PNL, oq.STRIKE, oq.WIN_PROB]
 
x = long_call

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- LONG CALL --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

x = long_put

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- LONG PUT --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
print(long_call[[oq.BID, oq.ASK, oq.PRICE, oq.STRIKE, oq.PROFIT, oq.PNL]])

print(long_put[[oq.BID, oq.ASK, oq.PRICE, oq.STRIKE, oq.PROFIT, oq.PNL]])

In [ ]:

#############################################################
#
#                  PICK SHORT CALL/PUT
#
############################################################

import sys

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.consts import optionQuote as oq
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_option_short

def dump_chain(chain):
    print('----------')    
    print(chain[[oq.EXP_DATE, oq.STRIKE, oq.PRICE, oq.PROFIT, oq.PNL, oq.WIN_PROB, oq.IMPLIED_VOLATILITY, oq.DELTA,oq.OPEN_INTEREST, oq.BID_ASK_SPREAD]])
 
symbol = 'MSFT'

target_price = get_price_history(symbol)['Close'].iloc[-1]

short_call = pick_option_short(symbol, 
                at.CALL,
                target_price,            
                min_annualized_return=20,
                min_price=0.5,
                min_pnl = 0,
                min_IV = 0.2,
                max_bid_ask_spread=5,
                max_days_to_expire=720, 
                min_open_interest=100)

short_put = pick_option_short(symbol, 
                at.PUT,
                target_price,  
                min_price=0.5,
                min_annualized_return=20,
                min_pnl = 0,
                min_IV = 0.2,
                max_bid_ask_spread=5,
                max_days_to_expire=90, 
                min_open_interest=100)

                            
print('#call', short_call.shape[0], ' #put ', short_put.shape[0])

dump_chain(short_call.sort_values(oq.EXP_DATE))
dump_chain(short_put.sort_values(oq.EXP_DATE))

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb
from tradingBot.settings.logConfig import LOGGING
import warnings
warnings.filterwarnings('ignore')

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)

features = [oq.DAYS_TO_EXPIRE, oq.PRICE, oq.PROFIT, oq.PNL, oq.STRIKE, oq.WIN_PROB]
 
x = short_call

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- LONG CALL --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

x = short_put

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- LONG PUT --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
print(short_call[short_call[oq.PROFIT] < 0][[oq.STRIKE, oq.PRICE, oq.PROFIT]])
print(short_put[short_put[oq.PROFIT] < 0][[oq.STRIKE, oq.PRICE, oq.PROFIT]])

In [ ]:
#############################################################
#
#                  CREDIT PUT SPREAD
#
############################################################

import sys
import numpy as np
import logging
sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_vertical_put_spreads
from tradingBot.settings.logConfig import LOGGING
LOGGING['loggers']['']['level'] = 'INFO'
logging.config.dictConfig(LOGGING)

symbol = 'MSFT'

target_price =get_price_history(symbol)['Close'].iloc[-1]

ps =  pick_vertical_put_spreads( symbol,                          
                            target_price,
                            credit=True,
                            exp_date_list=[],
                            spread_list=[],
                            min_price=np.nan,
                            max_spread = 10,
                            max_strike_ratio=0.25,
                            min_pnl=np.nan,                                
                            min_win_prob=np.nan,
                            max_bid_ask_spread=np.nan,
                            max_days_to_expire=np.nan, 
                            min_open_interest=np.nan)


print('credit put spread=', ps.shape[0])

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.EXP_DATE, oq.SPREAD, oq.PRICE, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.ANNUALIZED_RETURN]])

dump_chain(ps)

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb
from tradingBot.settings.logConfig import LOGGING
import warnings
warnings.filterwarnings('ignore')

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)

features = [oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.PNL, oq.WIN_PROB]
 
x = ps

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- CREDIT_PUT_SPREAD --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
ps.head(1).legs.values[0][1]

In [ ]:
features = [oq.SPREAD, oq.WIN_PROB, oq.PRICE, oq.PNL, oq.MAX_LOSS]
 
x = ps[ps[oq.PNL] > 0.5]

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- CREDIT PUT SPREAD --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
#############################################################
#
#                  CREDIT CALL SPREAD
#
############################################################
import sys
import numpy as np

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_vertical_call_spreads
from tradingBot.settings.logConfig import LOGGING
LOGGING['loggers']['']['level'] = 'INFO'
logging.config.dictConfig(LOGGING)

def dump_chain(exp_date, chain):
    print('----------')    
    print(chain[chain[oq.EXP_DATE]==exp_date][[oq.EXP_DATE, oq.STRIKE, oq.PNL, oq.IMPLIED_VOLATILITY, oq.DELTA, oq.OPEN_INTEREST, oq.BID_ASK_SPREAD]])
 
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

target_low_price =  current_price

target_high_price = current_price

cs = pick_vertical_call_spreads(symbol,                          
                            target_high_price,
                            credit=True,
                            min_short_price=0.5,
                            max_long_price=0.5,
                            max_spread = 10,
                            target_exp_date_list=[],
                            target_spread_list=[],
                            min_pnl=0.2,
                            min_short_IV=0.5,
                            max_long_IV=0.5,
                            min_win_prob=0,
                            min_annualized_return=0,
                            max_bid_ask_spread=30,
                            max_days_to_expire=90, 
                            min_open_interest=100)


print('credit call spread=', len(cs))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.EXP_DATE, oq.SPREAD, oq.P1_SK, oq.P1_LK, oq.PRICE, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.ANNUALIZED_RETURN]])
    
dump_chain(cs)

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')

features = [oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.PNL, oq.WIN_PROB]
 
x = cs

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- CREDIT_CALL_SPREAD --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
#############################################################
#
#                  CREDIT IRON CONDOR
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_iron_condor
from tradingBot.settings.logConfig import LOGGING
import logging
logging.getLogger("urllib3").setLevel(logging.WARNING)

LOGGING['loggers']['']['level'] = 'INFO'
logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

target_upper_price =  current_price + 10

target_lower_price = current_price + 5


cic = pick_iron_condor(symbol,                          
                        target_upper_price,
                        target_lower_price,
                        credit=True,
                        min_price= 0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0.0,
                        min_win_prob=50,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)

print('credit iron condor =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  DEBIT IRON CONDOR
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_iron_condor
from tradingBot.settings.logConfig import LOGGING
import logging
logging.getLogger("urllib3").setLevel(logging.WARNING)

LOGGING['loggers']['']['level'] = 'INFO'
logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

target_upper_price =  current_price + 5

target_lower_price = current_price - 5


cic = pick_iron_condor(symbol,                          
                        target_upper_price,
                        target_lower_price,
                        credit=False,
                        min_price= 0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0.5,
                        min_win_prob=50,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)

print('credit iron condor =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
cic[oq.STRATEGY]

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')

features = [oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.PNL, oq.WIN_PROB]
 
x = cic

print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- CREDIT_CALL_SPREAD --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')

x = cic

features = [oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.MAX_PROFIT, oq.PNL, oq.WIN_PROB, 'diff', oq.DAYS_TO_EXPIRE]
 
x['diff'] = (x[oq.P1_SK]-x[oq.P2_SK])

y = x[x['diff'] <= x[oq.SPREAD]]
print(y.shape[0])

x = y
print('count# ',cic.shape[0])
print(x.shape[0])
plt.subplots(figsize=(20,10))
print(symbol + '----- IRON CONDOR SPREAD --------')
for i, col in enumerate(features):
  plt.subplot(3,3,i+1)
  sb.distplot(x[col])
plt.show()

In [ ]:
#############################################################
#
#                  DEBIT IRON CONDOR
#
############################################################
import sys
import numpy as np

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_iron_condor

import logging

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)

def dump_chain(exp_date, chain):
    print('----------')    
    print(chain[chain[oq.EXP_DATE]==exp_date][[oq.EXP_DATE, oq.STRIKE, oq.PNL]])
 
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

target_upper_price =  current_price

target_lower_price = current_price

cic = pick_iron_condor(symbol,                          
                        target_upper_price,
                        target_lower_price,
                        credit=False,
                        min_short_price=0.5,
                        max_long_price=0.5,
                        max_spread = 10,
                        target_exp_date_list=[],
                        target_spread_list=[],
                        min_pnl=0,
                        min_short_IV=0.5,
                        max_long_IV=0.5,
                        min_win_prob=50,
                        min_annualized_return=0,
                        max_bid_ask_spread=30,
                        max_days_to_expire=90, 
                        min_open_interest=100)

print('credit iron condor =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  CREDIT CALL BUTTERFLY
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_call_butterfly

import logging

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

cic = pick_call_butterfly(symbol,                          
                        current_price,
                        credit=True,
                        min_price=0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0,
                        min_win_prob=0,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)


print('CREDIT CALL BUTTERFLY =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  DEBIT CALL BUTTERFLY
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_call_butterfly

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("urllib3").setLevel(logging.WARNING)
from tradingBot.settings.logConfig import LOGGING

import logging

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

cic = pick_call_butterfly(symbol,                          
                        current_price* 1.05,
                        credit=False,
                        min_price=0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0,
                        min_win_prob=0,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)


print('DEBIT CALL BUTTERFLY =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  CREDIT PUT BUTTERFLY
#
############################################################
import sys
import numpy as np

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_put_butterfly

import logging

from tradingBot.settings.logConfig import LOGGING

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

cic = pick_put_butterfly(symbol,                          
                        current_price,
                        credit=True,
                        min_price=0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0,
                        min_win_prob=0,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)


print('CREDIT PUT BUTTERFLY =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  DEBIT PUT BUTTERFLY
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_put_butterfly

import logging

from tradingBot.settings.logConfig import LOGGING

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

cic = pick_put_butterfly(symbol,                          
                        current_price,
                        credit=False,
                        min_price=0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0,
                        min_win_prob=0,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)


print('DEBIT PUT BUTTERFLY =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  IRON BUTTERFLY
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_iron_butterfly

import logging

from tradingBot.settings.logConfig import LOGGING

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

cic = pick_iron_butterfly(symbol,                          
                        current_price,
                        credit=True,
                        min_price=0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0,
                        min_win_prob=0,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)


print('IRON BUTTERFLY =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK, oq.P2_LT]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
#############################################################
#
#                  REVERSE IRON BUTTERFLY
#
############################################################
import sys
import numpy as np


sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.consts import asset as at
from tradingBot.utils.dataGetter import get_option_chain
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import pick_iron_butterfly

import logging

from tradingBot.settings.logConfig import LOGGING

LOGGING['loggers']['']['level'] = 'INFO'

logging.config.dictConfig(LOGGING)
symbol = 'MSFT'

current_price =get_price_history(symbol)['Close'].iloc[-1]

cic = pick_iron_butterfly(symbol,                          
                        current_price,
                        credit=False,
                        min_price=0,
                        max_spread = 10,
                        exp_date_list=[],
                        spread_list=[],
                        min_pnl=0,
                        min_win_prob=0,
                        min_annualized_return=0,
                        max_bid_ask_spread=10,
                        max_days_to_expire=90, 
                        min_open_interest=100)


print('REVERSE IRON BUTTERFLY =', len(cic))

from tradingBot.consts import optionQuote as oq
def dump_chain(chain):
    print('----------')    
    print(chain[[oq.DAYS_TO_EXPIRE, oq.SPREAD, oq.PRICE, oq.MAX_PROFIT, oq.MARGIN, oq.MAX_LOSS, oq.PNL, oq.WIN_PROB, oq.BREAKEVEN_L, oq.BREAKEVEN_H, oq.ANNUALIZED_RETURN, oq.P1_ST, oq.P1_SK, oq.P1_LK, oq.P2_ST, oq.P2_SK, oq.P2_LK, oq.P2_LT]])


if cic.shape[0] > 0:
    dump_chain(cic)

In [ ]:
cic.columns

In [ ]:
import sys
sys.path.append(r'\Users\jimhu\option_trader\src')

import sqlite3
from tradingBot.settings import app_settings  as settings    
from tradingBot.settings import schema as schema  

conn = sqlite3.connect(settings.DATA_ROOT_DIR+"/test.db")  

cur = conn.cursor()

In [ ]:
cur.execute("CREATE TABLE watchlist("+schema.watchlist+")")
cur.execute("CREATE TABLE user("+schema.user+")")
cur.execute("CREATE TABLE account("+schema.account+")")
cur.execute("CREATE TABLE strategy("+schema.strategy+")")
cur.execute("CREATE TABLE position("+schema.position+")")
cur.execute("CREATE TABLE transaction("+schema.transaction+")")


In [ ]:
transactions = "account_name TEXT,trx_time TEXT,symbol TEXT,type TEXT,buy_sell TEXT,open_close TEXT,\
    strike REAL,exp_date TEXT,quantity REAL,price REAL,commission REAL,fee REAL, amount REAL,\
    FOREIGN KEY(account_name) REFERENCES account(account_name)"
cur.execute("CREATE TABLE transactions("+transactions+")")

In [1]:
import sys
sys.path.append(r'\Users\jimhu\option_trader\src')

from tradingBot.utils import predictor
from tradingBot.utils.predictor import predict_price_range

x = predict_price_range('AAPL')

x

{'exp_date_list': ['2023-06-30',
  '2023-07-07',
  '2023-07-14',
  '2023-07-21',
  '2023-07-28',
  '2023-08-18',
  '2023-09-15',
  '2023-10-20',
  '2023-11-17',
  '2023-12-15',
  '2024-01-19',
  '2024-03-15',
  '2024-06-21',
  '2024-09-20',
  '2024-12-20',
  '2025-01-17',
  '2025-06-20',
  '2025-12-19'],
 '2023-06-30': {'high': 189.85999450683593,
  'low': 181.65999450683594,
  'win prob': 36.6},
 '2023-07-07': {'high': 191.41999450683593,
  'low': 180.09999450683594,
  'win prob': 38.7},
 '2023-07-14': {'high': 192.64999450683592,
  'low': 178.86999450683595,
  'win prob': 35.2},
 '2023-07-21': {'high': 193.50999450683594,
  'low': 178.00999450683594,
  'win prob': 35.3},
 '2023-07-28': {'high': 194.79999450683593,
  'low': 176.71999450683595,
  'win prob': 36.0},
 '2023-08-18': {'high': 198.20999450683593,
  'low': 173.30999450683595,
  'win prob': 40.8},
 '2023-09-15': {'high': 200.65999450683594,
  'low': 170.85999450683593,
  'win prob': 39.2},
 '2023-10-20': {'high': 203.34999450

In [3]:
import sys
sys.path.append(r'\Users\jimhu\option_trader\src')

from tradingBot.utils import predictor
from tradingBot.utils.predictor import predict_price_range

x = predict_price_range('AAPL', algo=predictor.PRICE_RANGE_BY_RANDOM_WALK, target_prob=40 )

x

{'2023-06-30': {'high': 190.45066714920708,
  'low': 181.16220302271807,
  'win prob': 40},
 '2023-07-07': {'high': 192.35993508737783,
  'low': 179.35511004756646,
  'win prob': 40},
 '2023-07-14': {'high': 194.28834343662876,
  'low': 177.566042824842,
  'win prob': 40},
 '2023-07-21': {'high': 195.2597851538119,
  'low': 176.67821261071776,
  'win prob': 40},
 '2023-07-28': {'high': 196.23608407958093,
  'low': 175.79482154766418,
  'win prob': 40},
 '2023-08-18': {'high': 199.19436757659108,
  'low': 173.1710618617126,
  'win prob': 40},
 '2023-09-15': {'high': 202.19724756710406,
  'low': 170.5864621170438,
  'win prob': 40},
 '2023-10-20': {'high': 206.2716233305782,
  'low': 167.20023565740593,
  'win prob': 40},
 '2023-11-17': {'high': 211.48024030444776,
  'low': 163.0618213465688,
  'win prob': 40},
 '2023-12-15': {'high': 212.53764150597,
  'low': 162.24651223983594,
  'win prob': 40},
 '2024-01-19': {'high': 215.74167301887758,
  'low': 159.82496276384236,
  'win prob': 40}